In [24]:
# !pip install keras tensorflow
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from tensorflow.keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential # Import Sequential from tensorflow.keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D # Import layers from tensorflow.keras
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # Import to_categorical from tensorflow.keras
import re


from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)
print(model.summary())

<ipython-input-24-1f3b17820baf>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-24-1f3b17820baf>:23: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 26s - 90ms/step - accuracy: 0.6449 - loss: 0.8270
144/144 - 3s - 23ms/step - accuracy: 0.6678 - loss: 0.7567
0.7566726803779602
0.6677588224411011
['loss', 'compile_metrics']


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 28, 128)        │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 196)            │       254,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,534,175 (5.85 MB)

 Trainable params: 511,391 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,022,784 (3.90 MB)

None


saved and executed the given model - above


In [25]:
import numpy as np

new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"]

# Preprocess the new text
new_text = [text.lower() for text in new_text]
new_text = [re.sub('[^a-zA-z0-9\s]', '', text) for text in new_text]
for i in range(len(new_text)):
    new_text[i] = new_text[i].replace('rt', ' ')

new_X = tokenizer.texts_to_sequences(new_text)
new_X = pad_sequences(new_X, maxlen=X.shape[1]) # Pad the sequences to match the input shape of the model

# Make predictions
prediction = model.predict(new_X)

# Get the predicted class (sentiment)
predicted_class = np.argmax(prediction, axis=1)

# Decode the predicted class using the LabelEncoder
predicted_sentiment = labelencoder.inverse_transform(predicted_class)

print(f"Predicted sentiment: {predicted_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted sentiment: ['Negative']


used the saved model(given) to predict on new text data (ex, “A lot of
good things are happening. We are respected again throughout the world, and that's a great
thing .@realDonaldTrump”)  above

In [39]:
!pip install scikeras

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, GridSearchCV
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier #This module is depreciated
from scikeras.wrappers import KerasClassifier #Import KerasClassifier from scikeras.wrappers instead
from tensorflow.keras.utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np

data = pd.read_csv('Data.csv')
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)


def createmodel(lstm_out=196, embed_dim=128):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

model = KerasClassifier(model=createmodel, verbose=0) #Pass the model as 'model' argument
param_grid = dict(lstm_out=[100,196], embed_dim=[64,128], model__batch_size=[32,64] ) # Example parameters and include batch size
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1) # n_jobs=-1 to use all processors

# Wrap the fit call within a try-except block to handle potential errors
try:
    grid_result = grid.fit(X_train, Y_train)
except AttributeError as e:
    # If AttributeError occurs, print the error message and suggest upgrading scikeras
    print(f"Error: {e}")
    print("This error might be due to compatibility issues between scikeras and sklearn.")
    print("Try upgrading scikeras using '!pip install scikeras --upgrade'")
except Exception as e: # Catch any other exceptions
    print(f"An unexpected error occurred: {e}")

# Check if grid_result is defined before using it
# if 'grid_result' in locals(): # Check if grid_result was successfully assigned
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
# else:
    # print("GridSearchCV failed to execute. Check the error messages above.")

<ipython-input-39-f88d08ec133b>:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-39-f88d08ec133b>:24: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')


Error: 'super' object has no attribute '__sklearn_tags__'
This error might be due to compatibility issues between scikeras and sklearn.
Try upgrading scikeras using '!pip install scikeras --upgrade'


GridSearchCV on the source code provided and produce the results with the best combination.---above
